In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms.flow.edmondskarp import edmonds_karp
from pathlib import Path
import time

--------------------- DEBUT --------------------------------------------------------            

In [ ]:
class FlowNet:
    """
    La classe FlowNet permet de crÃ©er et manipuler des graphes orientÃ©es avec 
    des flots et des capacitÃ©s.
    """
    def __init__(self, graph):  
        """
        contructeur de la classe FlowNet qui crÃ©er le graph en fonction du paramÃ¨tre donnÃ©
        Args:
            graph (string, Graph): soit un nom de fichier soit un Graph de la bilbiothÃ¨que NetworkX
        """
        self.reset()
        if isinstance(graph, nx.classes.graph.Graph):
            self.build_graph(graph)
        elif isinstance(graph, str):
            self.read_csv(graph)
        else:
            print("Le type de graphe fourni n'est pas reconnu\nnx.classes.graph.Graph OU string (nom de fichier)")
        
        self.R = None
        self.s = 's' # source
        self.t = 't' # puits
    
    def reset(self):
        """
        Met Ã  None les graph pour une nouvelle utilisation
        """
        self.R = None
        self.G = None
    
    def build_graph(self, graph):
        """
        crÃ©er un rÃ©seau Ã  partir dâ€™un graphe orientÃ© NetworkX 
        dans lequel les capacitÃ©s sont des attributs capacity des arcs
        Args:
            G (Graph): graph passÃ© au constructeur
        """
        self.G = graph
        
    def read_csv(self, path):
        """
        CrÃ©er un rÃ©seau Ã  partir de sa description dans un fichier CSV
        Args:
            path (string): chemin du fichier
        """
        p = Path(path)
        data = open(p, "r")
        print("file loaded : ", p)
        graphtype = nx.DiGraph()
        self.G = nx.parse_edgelist(data, delimiter=';', create_using=graphtype,
                              nodetype=str, data=(('capacity', int),))
        
    def export(self, name):
        """
        mÃ©thode qui permet l'export du graphe actuel en image au format png
        Args:
            name (string): nom du fichier
        """
        self.maj_flow_graph_g();
        pos = nx.random_layout(self.G)
        labels={ e : '{}|{}'.format(self.G[e[0]][e[1]]['flow'],self.G[e[0]][e[1]]['capacity']) for e in self.G.edges}
        node_colors=[ 'lightgrey' for _ in self.G.nodes() ]
        nx.draw_networkx(self.G, pos=pos, node_color=node_colors)
        nx.draw_networkx_edge_labels(self.G, pos=pos, edge_labels=labels)
        plt.savefig(name, format="PNG")
    
    def maj_flow_graph_g(self):
        """
        Comme nous ne travaillons que sur le graphe residuel 
        nous ne mettons pas Ã  jours ses flots lors des updates.
        La mÃ©thode est donc appelÃ©es avant l'export en PNG.
        """
        for e in self.G.edges:
            self.G[e[0]][e[1]]['flow'] = self.R[e[0]][e[1]]['flow']
            self.G[e[0]][e[1]]['capacity'] = self.R[e[0]][e[1]]['capacity']
        
    def show(self):
        """
        Affiche le graphe actuel
        """
        print("\n")
        for e in self.G.edges():
            print(e[0], "--->", e[1], " ", self.R[e[0]][e[1]]['flow'], "/",self.R[e[0]][e[1]]['capacity'])
      
    def capacity(self, s1, s2):
        """
        capacitÃ© d'un arc est un entier positif
        Args:
            s1 (char): sommet u 
            s2 (char): sommet v
        Returns:
            int: capacitÃ©
        """
        return self.G[s1][s2]["capacity"]
    def get_flow(self):
        """
        Renvoie le flot maximal
        Returns:
            [type]: [description]
        """
        return self.flow_value
    
    def update(self, s1, s2, c):
        """
        AprÃ¨s avoir calculÃ© le flot max, on mets Ã  jour la capacitÃ© de l'arc
        selon deux mÃ©thode diffÃ©rente en fonction d'une augmentation ou une 
        diminution de ce dernier.
        Args:
            s1 (char): sommet u 
            s2 (char): sommet v
            c (int): capacitÃ©
        """
        old = self.G[s1][s2]['capacity']
        self.G[s1][s2]['capacity'] = c
        self.R[s1][s2]['capacity'] = c
        
        # self.compute_max_flow() # pour verifier avec la mÃ©thode de base
        
        if (c <= old or old == 0): # si diminution de la capacitÃ© d'un arc
            diff = old - c
            self.worth_it(self.R, diff, '-', s1, s2)
        else: # si augmentation de la capacitÃ© d'un arc
            diff = c - old
            self.worth_it(self.R, diff, '+', s1, s2)
            
    def worth_it(self, R, diff, sign, s1, s2):
        """
        On ne recalcul le flot max seulement si l'arc passÃ© en paramÃ¨tre saturÃ©
        sinon il n'y a rien Ã  faire Ã  part mettre Ã  jour la capacitÃ© de l'arc
        Args:
            R (Graph): graphe rÃ©siduel
            diff (int): Ã©cart entre la capacitÃ© initiale et la nouvelle
            sign (char): permet de distinguer si augmentation ou diminution
            s1 (char): sommet u 
            s2 (char): sommet v
        """
        if sign == '-':
            # on fait le recalcul du flow en k itÃ©ration avec k = nouvelle capacitÃ©
            for k in range(diff): 
                self.compute_max_flow_decrease(R, 1, s1, s2)
        if sign == '+':
            # si l'arc est saturÃ© alors on recalcul le flot max
            if (R[s1][s2]['flow'] == R[s1][s2]['capacity']-diff): 
                self.compute_max_flow_increase(R)
    def compute_max_flow_decrease(self, R, diff, u, v):
        """
        Recalcul le flow max apres une diminution de la capacitÃ©
        Args:
            R (Graph): graphe rÃ©siduel
            diff (int): Ã©cart entre la capacitÃ© initiale et la nouvelle
            u (char): sommet u 
            v (char): sommet v
        """
        
        s = self.s
        t = self.t
        r_pred = R.pred
        r_succ = R.succ
        # on cherche un chemin simple de u Ã  s 
        _, pred1, _ = bidirectional_bfs(u, s, r_pred, r_succ)
        # on cherche un chemin simple de t Ã  v
        _, pred2, _ = bidirectional_bfs(t, v, r_pred, r_succ)
        
        if (R[u][v]['capacity'] < R[u][v]['flow']): # si l'arc changÃ© est saturÃ©
            R[u][v]['flow'] -= diff
            R[v][u]['flow'] += diff
        
            for arc in pred1:
                if (pred1[arc]): # si pas None
                    R[arc][pred1[arc]]["flow"] -= diff
                    R[pred1[arc]][arc]["flow"] += diff
            for arc in pred2:
                if (pred2[arc]): # si pas None
                    R[arc][pred2[arc]]["flow"] -= diff
                    R[pred2[arc]][arc]["flow"] += diff
                    
            self.flow_value -= diff
            self.compute_max_flow_increase(R)
    def compute_max_flow_increase(self, R):
        """[summary]
        Recalcul le flow max apres une augmentation de la capacitÃ©
        *Une partie du code ci-dessous provient de NetworkX
        Args:
            R (Graph): graphe rÃ©siduel
        """
        flow_val = self.get_flow()
        s = self.s
        t = self.t
        cutoff = float("inf")
        r_pred = R.pred
        r_succ = R.succ
        inf = R.graph["inf"]
        while flow_val < cutoff:
            v, pred, succ = bidirectional_bfs(s, t, r_pred, r_succ)
            if pred is None:
                break
            path = [v]
            u = v
            while u != s:
                u = pred[u]
                path.append(u)
            path.reverse()
            u = v
            while u != t:
                u = succ[u]
                path.append(u)
            flow_val += augment(path, r_succ, inf)
        self.flow_value = flow_val
        
    def compute_max_flow(self):
        """
        Calcul le folt max et crÃ©er le graph rÃ©siduel
        """
        self.R = edmonds_karp(self.G, "s", "t")
        self.flow_value = self.R.graph["flow_value"]
    
    def benchmark(self, s1, s2):
        """Ici nous ne faisons varier seulement la valeurs d'un seul arc
            mais cela montre quand mÃªme la diffÃ©rence de rapiditiÃ© entre
            la mÃ©thode base et "fines"
        Args:
            graph ([type]): [description]
            s1 ([type]): [description]
            s2 ([type]): [description]
            c ([type]): [description]
            vador ([type]): [description]
        """
        param = [(1,10000,1, "augmentation"), (10000, 1, -1, "diminution")]
        # compute_max_flow
        print("\nApproche avec mise a zero du residuel\nMoyenne de 10 iterations pour 10 000 update\n")
        for p in param:
            n = 10
            t = 0.0000000000000000
            start=p[0];end=p[1];step=p[2];way=p[3];
            for _ in range(1,n-1):
                st = time.time() 
                for i in range(start, end, step):  
                    self.G[s1][s2]['capacity'] = i
                    self.R[s1][s2]['capacity'] = i
                    self.compute_max_flow()
                fin = time.time()
                t += (fin - st)
            print(way, " temps = ",t/n)
        # update
        print("\nApproche avec utilisation du residuel\nMoyenne de 10 iterations pour 10 000 update\n")
        for p in param:
            n = 10
            t = 0.0000000000000000
            start=p[0];end=p[1];step=p[2];way=p[3];
            for _ in range(1,n-1):
                st = time.time() 
                for i in range(start, end, step):  
                    self.update(s1, s2, i)
                fin = time.time()
                t += (fin - st)
            print(way, " temps = ",t/n)
                
                
                
# --------------------- FIN --------------------------------------------------------


 ****** FROM NETWORKX ********<br>
 https://networkx.org/documentation/stable/_modules/networkx/algorithms/flow/edmondskarp.html#edmonds_karp<br>
 lÃ©gÃ©rement modifiÃ© <br>


In [ ]:
def augment(path, R_succ, inf):
    """Augment flow along a path from s to t.
    """
    # Determine the path residual capacity.
    flow = inf
    it = iter(path)
    u = next(it)
    for v in it:
        attr = R_succ[u][v]
        flow = min(flow, attr["capacity"] - attr["flow"])
        u = v
    if flow * 2 > inf:
        raise nx.NetworkXUnbounded("Infinite capacity path, flow unbounded above.")
    # Augment flow along the path.
    it = iter(path)
    u = next(it)
    for v in it:
        R_succ[u][v]["flow"] += flow
        R_succ[v][u]["flow"] -= flow
        u = v
    return flow
def bidirectional_bfs(s, t, R_pred, R_succ):
    """Bidirectional breadth-first search for an augmenting path.
    """
    pred = {s: None}
    q_s = [s]
    succ = {t: None}
    q_t = [t]
    while True:
        q = []
        if len(q_s) <= len(q_t):
            for u in q_s:
                for v, attr in R_succ[u].items():
                    if v not in pred and attr["flow"] < attr["capacity"]:
                        pred[v] = u
                        if v in succ:
                            return v, pred, succ
                        q.append(v)
            if not q:
                return None, None, None
            q_s = q
        else:
            for u in q_t:
                for v, attr in R_pred[u].items():
                    if v not in succ and attr["flow"] < attr["capacity"]:
                        succ[v] = u
                        if v in pred:
                            return v, pred, succ
                        q.append(v)
            if not q:
                return None, None, None
            q_t = q
"""
 ******** FROM NETWORKX ********   
"""

In [ ]:
N = FlowNet('fig42.csv')
N.compute_max_flow()
N.show()
print("Mise a jour de l'arc (d, c) de 7 a 5")
N.update('d', 'c', 5)
print("Flot max : ", N.get_flow())
print("Mise a jour de l'arc (d, c) de 5 a 9")
N.update('d', 'c', 9)
print("Flot max : ", N.get_flow())
N.reset()

In [ ]:
N = FlowNet('fig43.csv')
N.compute_max_flow()
N.show()
print("Mise a jour de l'arc (d, c) de 7 a 5")
N.update('d', 'c', 5)
print("Flot max : ", N.get_flow())
print("Mise a jour de l'arc (d, c) de 5 a 9")
N.update('d', 'c', 9)
print("Flot max : ", N.get_flow())
N.reset()

In [ ]:
G = nx.DiGraph()
G.add_edge("s", "a", capacity=16, flow=11)
G.add_edge("s", "b", capacity=13, flow=11)
G.add_edge("a", "c", capacity=12, flow=11)
G.add_edge("b", "a", capacity=4, flow=11)
G.add_edge("c", "b", capacity=9, flow=11)
G.add_edge("b", "d", capacity=14, flow=11)
G.add_edge("d", "c", capacity=7, flow=11)
G.add_edge("c", "t", capacity=20, flow=11)
G.add_edge("d", "t", capacity=4, flow=11)
N = FlowNet(G)
N.compute_max_flow()
N.show()
print("Flot max : ", N.get_flow())
print("Mise a jour de l'arc (a, c) de 12 a 10")
N.update('a', 'c', 10)
print("Flot max : ", N.get_flow())
print("Mise a jour de l'arc (a, c) de 10 a 14")
N.update('a', 'c', 14)
print("Flot max : ", N.get_flow())
N.reset()

In [ ]:
G = nx.DiGraph()
G.add_edge("s", "a", capacity=10)
G.add_edge("s", "b", capacity=8)
G.add_edge("a", "c", capacity=12)
G.add_edge("b", "a", capacity=4)
G.add_edge("c", "b", capacity=9)
G.add_edge("b", "d", capacity=12)
G.add_edge("d", "c", capacity=7)
G.add_edge("c", "t", capacity=18)
G.add_edge("d", "t", capacity=4)
N = FlowNet(G)
N.compute_max_flow()
print("Mise a jour de l'arc (s, b) de 8 a 9")
N.update('s', 'b', 9)
print("Flot max : ", N.get_flow())
print("Mise a jour de l'arc (s, b) de 9 a 20")
N.update('s', 'b', 20)
print("Flot max : ", N.get_flow())
N.export("graphDX.png")

In [ ]:
print("Ce test dure 1min20 en moyenne")
N.benchmark(s1='d', s2='c')
N.show()
print("Flot max : ", N.get_flow())
print("---------------------------------")
N.benchmark(s1='d', s2='c')
N.show()
print("Flot max : ", N.get_flow())